## <span style='color:#ff5f27'> 📝 Imports

In [1]:
# !pip install -r requirements.txt --quiet

In [2]:
import pandas as pd
import numpy as np
import great_expectations as ge
from great_expectations.core import ExpectationSuite, ExpectationConfiguration

from features.users import generate_users
from features.videos import generate_video_content
from features.interactions import generate_interactions

## <span style='color:#ff5f27'> ⚙️ Data Generation</span>


### <span style='color:#ff5f27'> 👥 Users Data Generation</span>

In [3]:
user_data = generate_users(25_000)

data_users_df = pd.DataFrame(user_data)
data_users_df.head()

/srv/hops/anaconda/envs/theenv/lib/python3.10/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,user_id,gender,age,country
0,VG365U,Female,44,Congo - Kinshasa
1,NT203T,Male,27,Taiwan
2,MX061G,Female,90,Bangladesh
3,ND779P,Female,60,Jamaica
4,QG530D,Other,84,Bangladesh


### <span style='color:#ff5f27'> 🎥 Content Data Generation</span>


In [4]:
# Generate data for 25_000 videos
video_data = generate_video_content(25_000, historical=True)

data_video_df = pd.DataFrame(video_data)
data_video_df.head()

/srv/hops/anaconda/envs/theenv/lib/python3.10/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,video_id,category,views,likes,video_length,upload_date
0,6QZ74E,Music,282723,71468,140,2023-06-13
1,2DQ39V,Sports,197,15,219,2022-10-05
2,4CD33T,News,12018,2589,70,2023-11-07
3,4EJ54U,Cooking,62837,14694,215,2022-12-15
4,3LX93C,Technology,91352,71148,205,2022-10-28


### <span style='color:#ff5f27'> 🔗 Interactions Generation</span>


In [5]:
num_interactions = 1_000_000

# Generate interactions
interactions = generate_interactions(num_interactions, user_data, video_data)

data_interactions_df = pd.DataFrame(interactions)
data_interactions_df.head()

/srv/hops/anaconda/envs/theenv/lib/python3.10/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,interaction_id,user_id,video_id,interaction_type,watch_time
0,0190-67-0484,XU795C,9RT36Y,like,117
1,9192-66-8799,MG416U,2FH04L,skip,157
2,0219-21-3726,WJ043C,3WF53F,like,95
3,2735-53-4930,BP920K,3TN51I,skip,214
4,6891-99-5386,LL440O,8IU90W,skip,103


## <span style="color:#ff5f27">👮🏻‍♂️ Great Expectations </span>

In [6]:
# Create a Great Expectations DataFrame from the pandas DataFrame
ge_users_df = ge.from_pandas(data_users_df)

# Initialize the expectation suite
expectation_suite_users = ge_users_df.get_expectation_suite()
expectation_suite_users.expectation_suite_name = "user_data_suite"

# Expectation: Age should be between 0 and 120
expectation_suite_users.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_between",
        kwargs={"column": "age", "min_value": 12, "max_value": 100}
    )
)

# Expectations: Columns should not have null values
for column in ge_users_df.columns:
    expectation_suite_users.add_expectation(
        ExpectationConfiguration(
            expectation_type="expect_column_values_to_not_be_null",
            kwargs={"column": column}
        )
    )

# Expectation: Gender should only contain specific values
expectation_suite_users.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_distinct_values_to_be_in_set",
        kwargs={"column": "gender", "value_set": ["Male", "Female", "Other"]}
    )
)

2024-04-03 12:43:12,096 INFO: 	0 expectation(s) included in expectation_suite.


/srv/hops/anaconda/envs/theenv/lib/python3.10/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{"kwargs": {"column": "gender", "value_set": ["Male", "Female", "Other"]}, "expectation_type": "expect_column_distinct_values_to_be_in_set", "meta": {}}

In [7]:
# Create a Great Expectations DataFrame from the pandas DataFrame
ge_video_df = ge.from_pandas(data_video_df)

# Initialize the expectation suite
expectation_suite_videos = ge_video_df.get_expectation_suite()
expectation_suite_videos.expectation_suite_name = "video_data_suite"

# Expectation: Views, Likes, and Video Length should be non-negative
for column in ["views", "likes", "video_length"]:
    expectation_suite_videos.add_expectation(
        ExpectationConfiguration(
            expectation_type="expect_column_values_to_be_between",
            kwargs={"column": column, "min_value": 0, "max_value": None}
        )
    )

# Expectation: Valid date format for upload_date
expectation_suite_videos.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_dateutil_parseable",
        kwargs={"column": "upload_date"}
    )
)

2024-04-03 12:43:13,931 INFO: 	0 expectation(s) included in expectation_suite.


{"kwargs": {"column": "upload_date"}, "expectation_type": "expect_column_values_to_be_dateutil_parseable", "meta": {}}

In [8]:
# Create a Great Expectations DataFrame from the pandas DataFrame
ge_interactions_df = ge.from_pandas(data_interactions_df)

# Initialize the expectation suite
expectation_suite_interactions = ge_interactions_df.get_expectation_suite()
expectation_suite_interactions.expectation_suite_name = "interactions_data_suite"

# Expectations: Non-null values in all columns
for column in ge_interactions_df.columns:
    expectation_suite_interactions.add_expectation(
        ExpectationConfiguration(
            expectation_type="expect_column_values_to_not_be_null",
            kwargs={"column": column}
        )
    )

# Expectation: Validate interaction types
expectation_suite_interactions.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_distinct_values_to_be_in_set",
        kwargs={
            "column": "interaction_type",
            "value_set": ['like', 'dislike', 'view', 'comment', 'share', 'skip']
        }
    )
)

# Expectation: Positive watch time
expectation_suite_interactions.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_between",
        kwargs={"column": "watch_time", "min_value": 0}
    )
)

2024-04-03 12:43:19,546 INFO: 	0 expectation(s) included in expectation_suite.


{"kwargs": {"column": "watch_time", "min_value": 0}, "expectation_type": "expect_column_values_to_be_between", "meta": {}}

## <span style="color:#ff5f27">🔮 Connect to Hopsworks Feature Store </span>

In [9]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://snurran.hops.works/p/11383
Connected. Call `.close()` to terminate connection gracefully.


## <span style="color:#ff5f27">🪄 Feature Group Creation </span>


In [10]:
users_fg = fs.get_or_create_feature_group(
    name="users",
    description="Users data.",
    version=1,
    primary_key=["user_id"],
    online_enabled=True,
    expectation_suite=expectation_suite_users,
)

users_fg.insert(data_users_df)
print('Done ✅')

Feature Group created successfully, explore it at 
https://snurran.hops.works/p/11383/fs/11331/fg/12301
2024-04-03 12:43:32,102 INFO: 	6 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://snurran.hops.works/p/11383/fs/11331/fg/12301


Uploading Dataframe: 0.00% |          | Rows 0/25000 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: users_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://snurran.hops.works/p/11383/jobs/named/users_1_offline_fg_materialization/executions
Done ✅


In [11]:
videos_fg = fs.get_or_create_feature_group(
    name="videos",
    description="Videos data.",
    version=1,
    primary_key=["video_id"],
    online_enabled=True,
    expectation_suite=expectation_suite_videos,
)

videos_fg.insert(data_video_df)
print('Done ✅')

Feature Group created successfully, explore it at 
https://snurran.hops.works/p/11383/fs/11331/fg/12302
2024-04-03 12:43:44,451 INFO: 	4 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://snurran.hops.works/p/11383/fs/11331/fg/12302


Uploading Dataframe: 0.00% |          | Rows 0/25000 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: videos_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://snurran.hops.works/p/11383/jobs/named/videos_1_offline_fg_materialization/executions
Done ✅


In [12]:
interactions_fg = fs.get_or_create_feature_group(
    name="interactions",
    description="Interactions data.",
    version=1,
    primary_key=["interaction_id", "user_id", "video_id"],
    online_enabled=True,
    expectation_suite=expectation_suite_interactions,
)

interactions_fg.insert(data_interactions_df)
print('Done ✅')

Feature Group created successfully, explore it at 
https://snurran.hops.works/p/11383/fs/11331/fg/12303
2024-04-03 12:43:49,670 INFO: 	7 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://snurran.hops.works/p/11383/fs/11331/fg/12303


Uploading Dataframe: 0.00% |          | Rows 0/1000000 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: interactions_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://snurran.hops.works/p/11383/jobs/named/interactions_1_offline_fg_materialization/executions
Done ✅


## <span style="color:#ff5f27">🪄 Ranking Feature Group </span>


In [13]:
video_interactions_df = pd.merge(
    data_interactions_df, 
    data_video_df, 
    on='video_id', 
    how='inner',
)

ranking_df = pd.merge(
    video_interactions_df, 
    data_users_df, 
    on='user_id', 
    how='inner',
)

ranking_df['label'] = np.where(
    ranking_df.interaction_type.isin(['view', 'like', 'share', 'comment']), 
    1, 
    0,
)

ranking_df.drop(
    ['interaction_id', 'interaction_type', 'watch_time'], 
    axis=1, 
    inplace=True,
)

ranking_df.head()

,user_id,video_id,category,views,likes,video_length,upload_date,gender,age,country,label
0,XU795C,9RT36Y,Entertainment,1394,1177,117,2022-04-13,Female,22,Ceuta & Melilla,1
1,XU795C,2DP12B,Entertainment,248949,135612,21,2023-11-01,Female,22,Ceuta & Melilla,0
2,XU795C,4DX22W,Travel,158994,69318,50,2023-06-01,Female,22,Ceuta & Melilla,0
3,XU795C,5JJ22I,Comedy,310042,17216,117,2024-03-26,Female,22,Ceuta & Melilla,0
4,XU795C,1RH45E,Comedy,15541,8175,35,2023-04-25,Female,22,Ceuta & Melilla,1


In [14]:
ranking_fg = fs.get_or_create_feature_group(
    name="ranking",
    description="Ranking Data.",
    version=1,
    primary_key=["user_id", "video_id"],
    online_enabled=True,
)

ranking_fg.insert(ranking_df)
print('Done ✅')

Feature Group created successfully, explore it at 
https://snurran.hops.works/p/11383/fs/11331/fg/12304


Uploading Dataframe: 0.00% |          | Rows 0/1002083 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: ranking_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://snurran.hops.works/p/11383/jobs/named/ranking_1_offline_fg_materialization/executions
Done ✅


---